# Exploring National Water Model (NWM) operational streamflow output using xarray

This notebook demonstrates how to open and explore a National Water Model NetCDF file using xarray.

## Use `requests` to download a file

This function will download a file and save it, without overwriting.

In [1]:
from pathlib import Path
import requests

def download_file(url: str, filename: str) -> None:
    """Download the file at url and save to filename.

    Args:
        url (str): URL path to file for download.
        filename (str): Destination to save downloaded file.

    Returns:
        None.
    """
    # Return if file exists
    if Path(filename).exists():
        return

    # Download file and write 
    with open(filename, "wb") as fo:
        response = requests.get(url)
        fo.write(response.content)

## Download a NWM NetCDF file

The next step is to use the `download_file` funtion, defined above, to retrieve a NWM NetCDF file. In this case, we retrieve a single file from Google Cloud Platform. The file contains the streamflow (`channel_rt`) values for all reaches in the CONUS domain (`conus`) for the first time step (`tm00`) for the un-nudged extended analysis and assimilation cycle (`analysis_assim_extend_no_da`) that was issued at 16Z on 2024-05-01.

You can browse the avialable NWM NetCDF file archive on Google Cloud Platform here:
https://console.cloud.google.com/marketplace/product/noaa-public/national-water-model

In [2]:
# Inidicate filename and build url
filename = "nwm.t16z.analysis_assim_extend_no_da.channel_rt.tm00.conus.nc"
url = "/".join((
    "https://storage.googleapis.com",
    "national-water-model",
    "nwm.20240501",
    "analysis_assim_extend_no_da",
    filename
    ))

# Download file
download_file(url, filename)

## Open NWM NetCDF file

Opening a NetCDF file for exploration and analysis using `xarray` is very easy.

In [3]:
import xarray as xr

# Lazily open dataset
ds = xr.open_dataset(filename)

## Attributes

NWM NetCDF `channel_rt` data are indexed by `time`, `reference_time` (time of issuance), and `feature_id` (NHD comid). These files also contain a number of variables in addition to streamflow.

|Variable|Description|
|-|-|
|crs|Stub variable used to store CRS definition for channel reach centroid coordinates.|
|streamflow|River flow in $m^3 s^{-1}$.|
|nudge|Amount of stream flow alteration due to assimilation in $m^3 s^{-1}$.|
|velocity|Average river velocity in $m s^{-1}$.|
|qSfcLatRunoff|Lateral inflow from terrain routing in $m^3 s^{-1}$.|
|qBucket|Flux from groundwater bucket in $m^3 s^{-1}$.|

An overview of the dataset header is available by just printing the dataset itself. Metadata are embeded in the dataset and accessible using the `attrs` attribute (i.e. `ds.attrs`). Metadata for individual variables are accessible in the same way using dot indexing (i.e. `ds.streamflow.attrs`).

In [4]:
ds

<xarray.Dataset> Size: 133MB
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776734)
Coordinates:
  * time            (time) datetime64[ns] 8B 2024-05-01T16:00:00
  * reference_time  (reference_time) datetime64[ns] 8B 2024-04-30T12:00:00
  * feature_id      (feature_id) int64 22MB 101 179 ... 1180001803 1180001804
Data variables:
    crs             |S1 1B ...
    streamflow      (feature_id) float64 22MB ...
    nudge           (feature_id) float64 22MB ...
    velocity        (feature_id) float64 22MB ...
    qSfcLatRunoff   (feature_id) float64 22MB ...
    qBucket         (feature_id) float64 22MB ...
Attributes: (12/19)
    TITLE:                      OUTPUT FROM NWM v3.0
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2024-04-30_12:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2024-05-01_16:00:00
    ...                         ...
    model_configuration:        analysis_and_assimilation
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           1
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

In [5]:
ds.attrs

{'TITLE': 'OUTPUT FROM NWM v3.0',
 'featureType': 'timeSeries',
 'proj4': '+proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0 +lat_2=60.0 +lat_0=40.0 +lon_0=-97.0 +x_0=0 +y_0=0 +k_0=1.0 +nadgrids=@',
 'model_initialization_time': '2024-04-30_12:00:00',
 'station_dimension': 'feature_id',
 'model_output_valid_time': '2024-05-01_16:00:00',
 'model_total_valid_times': 28,
 'stream_order_output': 1,
 'cdm_datatype': 'Station',
 'Conventions': 'CF-1.6',
 'code_version': 'v5.3.0-alpha1',
 'NWM_version_number': 'v3.0',
 'model_output_type': 'channel_rt',
 'model_configuration': 'analysis_and_assimilation',
 'dev_OVRTSWCRT': 1,
 'dev_NOAH_TIMESTEP': 3600,
 'dev_channel_only': 1,
 'dev_channelBucket_only': 0,
 'dev': 'dev_ prefix indicates development/internal meta data'}

In [6]:
ds.streamflow.attrs

{'long_name': 'River Flow',
 'units': 'm3 s-1',
 'grid_mapping': 'crs',
 'valid_range': array([      0, 5000000], dtype=int32)}

## Selecting data

Subsetting and transforming the dataset can be accomplished using the `sel` and `to_dataframe` methods.

In [7]:
# Subset the data to two reaches using sel
subset = ds.sel(feature_id=[101, 179])
subset

<xarray.Dataset> Size: 113B
Dimensions:         (time: 1, reference_time: 1, feature_id: 2)
Coordinates:
  * time            (time) datetime64[ns] 8B 2024-05-01T16:00:00
  * reference_time  (reference_time) datetime64[ns] 8B 2024-04-30T12:00:00
  * feature_id      (feature_id) int64 16B 101 179
Data variables:
    crs             |S1 1B ...
    streamflow      (feature_id) float64 16B ...
    nudge           (feature_id) float64 16B ...
    velocity        (feature_id) float64 16B ...
    qSfcLatRunoff   (feature_id) float64 16B ...
    qBucket         (feature_id) float64 16B ...
Attributes: (12/19)
    TITLE:                      OUTPUT FROM NWM v3.0
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2024-04-30_12:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2024-05-01_16:00:00
    ...                         ...
    model_configuration:        analysis_and_assimilation
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           1
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

In [8]:
# Subset the streamflow only
streamflow_subset = ds.streamflow.sel(feature_id=[101, 179])
streamflow_subset

<xarray.DataArray 'streamflow' (feature_id: 2)> Size: 16B
[2 values with dtype=float64]
Coordinates:
  * feature_id  (feature_id) int64 16B 101 179
Attributes:
    long_name:     River Flow
    units:         m3 s-1
    grid_mapping:  crs
    valid_range:   [      0 5000000]

In [9]:
# Convert to pandas.DataFrame
# https://pandas.pydata.org/
df = ds.sel(feature_id=[101, 179]).to_dataframe()
df

crs  streamflow  nudge  \
time                reference_time      feature_id                           
2024-05-01 16:00:00 2024-04-30 12:00:00 101         b''       14.66    0.0   
                                        179         b''        0.00    0.0   

                                                    velocity  qSfcLatRunoff  \
time                reference_time      feature_id                            
2024-05-01 16:00:00 2024-04-30 12:00:00 101             0.31            0.0   
                                        179             0.01            0.0   

                                                    qBucket  
time                reference_time      feature_id           
2024-05-01 16:00:00 2024-04-30 12:00:00 101         0.45123  
                                        179         0.00007